In [1]:
import torch

s=1

Nif=4
Nix=4
Niy=4

Nkx=2
Nky=2

Nox=int((Nix-Nkx)/s+1)
Noy=int((Niy-Nky)/s+1)
Nof=4

Nif=Nof


nof_par=4
nif_par=4
nkx_par=2
nky_par=2
kernel_par=Nkx*Nky

cycle=0

input=torch.randn(Nif, Nix, Niy)
weight=torch.randn(Nif, Nkx, Nky)
output=torch.zeros(Nof, Nox, Noy)


In [2]:
#dw without parallelism
for no in range(Nof): 
    for y in range(Noy):
        for x in range(Nox):
            #for ni in range (Nif): 
            for ky in range(Nky):
                for kx in range(Nkx):
                    #print(no+1, x+3*y)
                    #print(no+1, s*x+kx, s*y+ky)
                    #print(no+1, kx+2*ky)
                    #print('\n')
                    output[no][x][y]+=input[no][s*x+kx][s*y+ky] * weight[no][kx][ky]
                    cycle+=1


print(cycle)

144


In [3]:
print(output)

tensor([[[ 0.0810, -0.2652, -2.1615],
         [ 1.9637, -0.9055, -0.7595],
         [-0.4331, -0.7083, -0.0434]],

        [[ 2.1409, -2.4707,  1.5263],
         [-0.2562,  1.2621, -0.8949],
         [ 1.6921, -1.5039,  2.2163]],

        [[-5.2274,  0.5256,  2.0100],
         [ 0.3335, -0.2919, -1.1516],
         [ 3.2920, -4.6294, -4.5473]],

        [[ 0.8894,  1.2830, -2.3817],
         [ 1.2622, -1.0718, -0.2295],
         [ 3.8316, -2.2469,  3.7432]]])


In [5]:
#dw with parallelism
cycle=0
for no in range(Nof//nof_par): 
    for y in range(Noy):
        for x in range(Nox):
            #for ni in range (Nif): 
            for ky in range(Nky//nky_par):
                for kx in range(Nkx//nkx_par):
                    #in one cycle
                    for no1 in range(nof_par):
                        for ky1 in range(nky_par):
                            for kx1 in range(nkx_par):
                                output[no*nof_par+no1][x][y]+=input[no*nof_par+no1][s*x+(kx*nkx_par+kx1)][s*y+(ky*nky_par+ky1)] * weight[no*nof_par+no1][(kx*nkx_par+kx1)][(ky*nky_par+ky1)]
                    cycle+=1
print(cycle)

9


In [23]:
print(output)

tensor([[[-1.1731,  3.3708, -5.6262, -9.8522],
         [ 5.5946, -2.9809, -3.1357,  3.6884],
         [ 1.5248, -7.7382, -4.1389,  2.9927],
         [-0.9772,  0.3627,  8.3034, -4.4183]],

        [[-2.3049, -3.5639,  4.9046,  3.0635],
         [ 2.9943, -6.8540, -7.7821,  2.5723],
         [ 2.8449, -4.2599,  1.5922, -1.1001],
         [-1.5686,  3.5773,  5.1814, -2.7355]],

        [[ 1.7156,  5.2450,  4.4732, -0.7336],
         [-7.1752, -3.0907,  2.6625,  5.3353],
         [-4.5596, -0.2805,  1.6685,  2.5080],
         [-0.7932, -2.9263,  0.6053,  3.3063]]])
